In [6]:
import numpy as np
from RandomForest import RandomForest
from EsembleNNet import EsembleNNet
from SOMAC import SOMAC

In [7]:
train_file = "/Users/andre.gomes/Temp/SOMAC-ML/data/14092018/somac/training_file.npy"
test_file = "/Users/andre.gomes/Temp/SOMAC-ML/data/14092018/somac/backlog_file.npy"

In [8]:
test_data = np.load(test_file, encoding = "latin1").item()

In [9]:
# somac = SOMAC(reg_csma = RandomForest(), reg_tdma = EsembleNNet())
# EsembleNNet(n_new_estimators = 10, n_neurons = 3)
somac = SOMAC(reg_csma = EsembleNNet(n_new_estimators = 10, n_neurons = 3, bag_size = 1),
              reg_tdma = EsembleNNet(n_new_estimators = 10, n_neurons = 3, bag_size = 1))
somac.train(train_file)

In [10]:
for t in range(80):
    somac.decision(test_data[t])
    y = test_data[t]["metrics"][0, 1]
    prot = test_data[t]["prot"]
    print("---")

Prot: 0, y = 0.16666649281978607
y_hat_CSMA = 17.99, y_hat_TDMA = 1.02
Evaluation: v_csma = 17.99, v_tdma = 1.02
---
Prot: 0, y = 0.2758620008826256
y_hat_CSMA = 12.7, y_hat_TDMA = 2.69
Evaluation: v_csma = 15.64, v_tdma = 6.85
---
Prot: 0, y = 0.1189655028283596
y_hat_CSMA = 35.84, y_hat_TDMA = 2.45
Evaluation: v_csma = 38.87, v_tdma = 7.69
---
Prot: 0, y = 0.05000000074505806
y_hat_CSMA = 36.83, y_hat_TDMA = 0.04
Evaluation: v_csma = 39.77, v_tdma = 5.93
---
Prot: 0, y = 0.03333349898457527
y_hat_CSMA = 43.52, y_hat_TDMA = 6.4
Evaluation: v_csma = 46.35, v_tdma = 12.74
---
Prot: 0, y = 0.1333335041999817
y_hat_CSMA = 43.81, y_hat_TDMA = 6.43
Evaluation: v_csma = 46.54, v_tdma = 13.12
---
Prot: 0, y = 0.20000000298023224
y_hat_CSMA = 43.69, y_hat_TDMA = 6.46
Evaluation: v_csma = 46.33, v_tdma = 13.44
---
Prot: 0, y = 0.4166665133088827
y_hat_CSMA = 49.36, y_hat_TDMA = 13.32
Evaluation: v_csma = 51.9, v_tdma = 20.53
---
Prot: 0, y = 0.1333335041999817
y_hat_CSMA = 47.71, y_hat_TDMA = 1

---
Prot: 0, y = 30.40000033378601
y_hat_CSMA = 25.69, y_hat_TDMA = 19.73
Evaluation: v_csma = 27.01, v_tdma = 22.84
---
Prot: 0, y = 31.43333339691162
y_hat_CSMA = 25.68, y_hat_TDMA = 19.5
Evaluation: v_csma = 27.0, v_tdma = 22.62
---
Prot: 0, y = 31.016666412353516
y_hat_CSMA = 25.48, y_hat_TDMA = 19.58
Evaluation: v_csma = 26.79, v_tdma = 22.7
---
Prot: 0, y = 33.120115637779236
y_hat_CSMA = 25.79, y_hat_TDMA = 23.53
Evaluation: v_csma = 27.08, v_tdma = 26.65
---
Prot: 0, y = 34.325862765312195
y_hat_CSMA = 26.15, y_hat_TDMA = 23.9
Evaluation: v_csma = 27.44, v_tdma = 27.03
---
Prot: 0, y = 31.27988526970148
y_hat_CSMA = 26.7, y_hat_TDMA = 24.62
Evaluation: v_csma = 27.98, v_tdma = 27.75
---
Prot: 0, y = 31.33333334326744
y_hat_CSMA = 26.31, y_hat_TDMA = 23.48
Evaluation: v_csma = 27.58, v_tdma = 26.62
---
Prot: 0, y = 31.488506078720093
y_hat_CSMA = 26.25, y_hat_TDMA = 23.77
Evaluation: v_csma = 27.51, v_tdma = 26.92
---
Prot: 0, y = 30.033333629369736
y_hat_CSMA = 26.45, y_hat_TDM